# Scalar energy function of two parts of series beeing the same or not
- we want two effects here: If the sampling rate is different they should still be the same
- also if the noise is within some level we want similarity
- also on small ablation/missing data we want similarity

- we want dissimilarity on different sections of the signal
- we want dissimilarity with different signals
- we want dissimilarity when the signal noise is too large

Some pointers for implementation:
https://github.com/pietz/simclr/blob/master/SimCLR.ipynb
https://github.com/Spijkervet/SimCLR

In [1]:
import utils as ut
import pickle
import copy
import torch
from torch import nn

---
# Data Generation

The data we generate is generated time series with different characteristics (peaks,
square signal, sinus, spikes). We simulate four sensors with different noise and
sampling rate, and each senses a different signal but with the same basic characteristics.

In [2]:
ts_length = 360000 # in 100Hz, so let's generate 1h of signals each
sensor1 = ut.Sensor(40, 0.0, 0.2)
sensor2 = ut.Sensor(80, 0.1, 0.1)
sensor3 = ut.Sensor(80, 0.0, 0.3)
sensor4 = ut.Sensor(20, 0.0, 0.1)

signal1, signal1_noise = sensor1.sense_signal(ts_length)
signal2, signal2_noise = sensor2.sense_signal(ts_length)
signal3, signal3_noise = sensor3.sense_signal(ts_length)
signal4, signal4_noise = sensor4.sense_signal(ts_length)


data with only two columns is assumed to have no missing value field, just return the whole index
data with only two columns is assumed to have no missing value field, just return the whole index
data with only two columns is assumed to have no missing value field, just return the whole index
data with only two columns is assumed to have no missing value field, just return the whole index


In a first step we need to get them to the sampe sampling rate, to always have the same amount of time steps

In [5]:
s1_100hz = ut.AugmentTSSignal(signal1_noise).expand(sensor1.sampling_rate, 100).data
s2_100hz = ut.AugmentTSSignal(signal2_noise).expand(sensor2.sampling_rate, 100).data
s3_100hz = ut.AugmentTSSignal(signal3_noise).expand(sensor3.sampling_rate, 100).data
s4_100hz = ut.AugmentTSSignal(signal4_noise).expand(sensor4.sampling_rate, 100).data

assert s1_100hz.shape[0] == s2_100hz.shape[0]
assert s2_100hz.shape[0] == s3_100hz.shape[0]
assert s3_100hz.shape[0] == s4_100hz.shape[0]

In [6]:
# since that data generation needs w bit, let's save it for later
with open('data/signal_data.pkl', 'wb') as file:
    pickle.dump((s1_100hz, s2_100hz, s3_100hz, s4_100hz), file)

In [ ]:
# in case we manually want to load the data from above, execute this cell:
with open('data/signal_data.pkl', 'rb') as file:
    s1_100hz, s2_100hz, s3_100hz, s4_100hz = pickle.load(file)

# Representation Learning

We can now try to learn contrastive representations where the 4 different aspects of
the signal are near to each other independent of the noise and sampling rate. In the easiest way
we just use a 1D convolutional encoder, and apply the loss.

A question is how the get batches and data. What we want is to take random segments of the time series and augment it.
Additionally we take other random segments of the same time series and claim this is different. But since we have only
for modes in the time series, this is actually not that clear that you pick in fact a chunk with a different mode.

In [ ]:
# we have 100Hz data
# it depends on the target domain what good windows are.
# In our case we can say that most signals need 4 seconds to do a full 2Pi cycle
# so maybe encoding approx a 1Pi cycle is a good idea, that is 2s, aka 200 time steps

cos = nn.CosineSimilarity(dim=1, eps=1e-6)

def base_dict(N):
    all_js = {j: 0.0 for j in range(2*N)}
    all_sims = {i: all_js.deepcopy()  for i in range(2*N)}
    return all_sims

def loss(i, j, all_similarities, temperature_param):
    ze = all_similarities
    T = temperature_param
    vplus = (torch.exp(ze[i][j]/T))
    vminus = 0 #TODO
    l = -torch.log(vplus / vminus)
    return l

def train_mini(mini_batch, temperature_param, augmentations, network, projection):
    all_pairs = []
    N = len(mini_batch)
    for x in mini_batch:
        tau1 = augmentations.random_choice()
        tau2 = augmentations.random_choice()
        x_1 = tau1(x)
        x_2 = tau2(x)
        o_1 = network(x_1)
        o_2 = network(x_2)
        p_1 = projection(o_1)
        p_2 = projection(o_2)
        all_pairs.append(p_1)
        all_pairs.append(p_2)
    all_similarities = base_dict(N)
    for i in range(2*N):
        for j in range(2*N):
            p_1 = all_pairs[i]
            p_2 = all_pairs[j]
            all_similarities[i][j] = cos(p_1, p_2)
    l = ((1/2)*N)*
# pseudo code:
# for epochs:
#     sample N random samples
#     for each n in N:
#         get two random augmentations
#         augment n into n_1 and n2
#         run them trough network to get output_1, output_2 (with extra layer)
#     we now have 2N outputs
#     compute similarity for all examples
#     compute loss for each positive pair
#     avarage all losses
#     backprob





